In [ ]:
import re
from mwclient import Site
from mwclient import page




In [150]:
def gene_to_snp_texts(site, gene_name):
    snp_name_to_raw_text = {}
    gene_page = page.Page(site,gene_name)
    snp_category_str = "Category:Is a snp"
    for linked_page in gene_page.links():
        if snp_category_str in [c.name for c in linked_page.categories()]:
            snp_name = linked_page.name
            raw_text = linked_page.text()
            snp_name_to_raw_text[snp_name] = raw_text
    return(snp_name_to_raw_text)


In [151]:
def remove_with_regex(pattern, text):
    matches = re.findall(pattern, text)
    for match in matches:
        text = text.replace(match[0], "")
    return(text.strip())



In [152]:
def clean_raw_page_text(text):
    
    # Remove all newline characters from the text
    text = v.replace("\n","").replace("\r","").replace("\t","")
    
    # Remove all the text that is inside of double curly braces, i.e from tables.
    text = remove_with_regex(r'(\{\{(.|\n)*?\}\})', text)
        
    # Remove the square brackets that indicate links, but keep the interior strings that aren't links.
    text = remove_with_regex(r'((www|http:|https:)+[^\s]+[\w])', text)
    text = text.replace("[","").replace("]","")
    return(text.strip())

In [154]:

site = mwclient.Site('bots.snpedia.com', path="/")


for k,v in gene_to_snp_texts(site, "CCR5").items():
    print()
    print(k)
    print(clean_raw_page_text(v))



I3003626
The public name for this SNP is rs333, and it is most commonly known as the CCR5 Delta32 (or 'Delta 32') mutation.It is discussed in numerous articles and blogs, including  here and / here.*II +/+: Not resistant to HIV infection; shows average time of progression to AIDS after infection.*DI Delta32/+: Not resistant to HIV infection but may have slower progression to AIDS after infection.*DD Delta32/Delta32: Resistant to infection by the most common strain of HIV people usually encounter, though protection is not complete.Resistance to HIV/AIDS

Rs1799987
CCR5 59029G, also known as -2459 PMID 9742978, PMID 12815099

Rs1799988


Rs1800452
Optimization of candidate-gene SNP-genotyping by flexible oligonucleotide microarrays; analyzing variations in immune regulator genes of hay-fever samples.

Rs1800560


Rs1800940


Rs1800944


Rs2856758
Rs2856758 is a SNP in the CCR5 gene (CC-chemokine receptor type 5). Variants in CCR5 are associated with resistance to HIV infection.  Another